In [3]:
import numpy as np

# Load data

In [2]:
load_raw_data = False
if load_raw_data:
    brain_masks = np.load('../data/all_2016_2017/brain_masks.npy')
    clinical_inputs = np.load('../data/all_2016_2017/clinical_inputs.npy')
    ids = np.load('../data/all_2016_2017/ids.npy')
    included_subjects = np.load('../data/all_2016_2017/included_subjects.npy') # empty
    params = np.load('../data/all_2016_2017/params.npy') # inaccessible

    # Heavy
    lesion_GT = np.load('../data/all_2016_2017/lesion_GT.npy')
    ct_inputs = np.load('../data/all_2016_2017/ct_inputs.npy')

# Visualisation

In [6]:
ls

 Le volume dans le lecteur D s'appelle Data
 Le num‚ro de s‚rie du volume est DC28-9704

 R‚pertoire de D:\GitHub\StrokeLesionPredict-BIO503\PerfusionCT-Net

10/06/2020  12:05    <DIR>          .
10/06/2020  12:05    <DIR>          ..
01/06/2020  12:46             1ÿ985 .gitignore
09/06/2020  19:21    <DIR>          .ipynb_checkpoints
09/06/2020  13:56             5ÿ314 augmentation_vis.py
09/06/2020  14:25    <DIR>          checkpoints
09/06/2020  14:06    <DIR>          configs
01/06/2020  22:29    <DIR>          dataio
01/06/2020  12:46             1ÿ558 journal.md
08/06/2020  15:20    <DIR>          models
10/06/2020  12:05             8ÿ806 Quentin Visual Tools.ipynb
08/06/2020  15:20             1ÿ335 README.md
08/06/2020  15:20               381 requirements.txt
08/06/2020  15:25               648 requirements_gsp.txt
09/06/2020  19:24    <DIR>          src
01/06/2020  22:29    <DIR>          static
08/06/2020  15:25             6ÿ709 train_segmentation.py
09/06/2020  19:27     

In [6]:
!python train_segmentation.py -c D:/GitHub/StrokeLesionPredict-BIO503/PerfusionCT-Net/configs/small_dataset_config_unet_pct_multi_att_dsv.json

^C


In [7]:
from utils import utils

In [9]:
import torchio
print(torchio.__file__)

d:\python\anaconda3\envs\pctnet\lib\site-packages\torchio\__init__.py


In [10]:
from models import get_model

In [14]:
from utils.utils import json_file_to_pyobj

In [16]:
get_dataset('gsd_pCT')

dataio.loaders.geneva_stroke_dataset_pCT.GenevaStrokeDataset_pCT

In [ ]:
import os
import random

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from torch.utils.data import DataLoader
from tqdm import tqdm

from dataio.loaders import get_dataset
from dataio.transformation import get_dataset_transformation

from models import get_model
from utils import utils
from utils.utils import json_file_to_pyobj


import torchio

print(torchio.__file__)

"""
Notes:
shift is working
rotation is working
noise is working

rot + shift takes wrong interpolation for mask
skewing introduces weird artifacts, same with scaling
"""


# todo images need to be in form X,Y,Z
def aug_vis(json_filename):
    # Visualisation arguments
    with_mask = True
    len_x = 5  # number of images on x-axis for vis pdf
    len_y = 5  # number of images on y-axis for vis pdf
    nbr_pages = 10
    total_images = len_x * len_y * nbr_pages  # total number of slices that will be augmented

    # Load options
    json_opts = json_file_to_pyobj(json_filename)
    train_opts = json_opts.training

    # Architecture type
    arch_type = train_opts.arch_type

    # Setup Dataset and Augmentation
    ds_class = get_dataset('gsd_pCT')
    ds_path = json_opts.data.data_dir
    template_path = json_opts.data.template_dir
    ds_transform = get_dataset_transformation('mlebe', opts=json_opts.augmentation,
                                              max_output_channels=json_opts.model.output_nc)

    # Setup channels
    channels = json_opts.data_opts.channels
    if len(channels) != json_opts.model.input_nc \
            or len(channels) != getattr(json_opts.augmentation, 'mlebe').scale_size[-1]:
        raise Exception(
            'Number of data channels must match number of model channels, and patch and scale size dimensions')

    # Setup Data Loader
    split_opts = json_opts.data_split
    train_dataset = ds_class(template_path, ds_path, json_opts.data, split='train',
                             transform=ds_transform['train'],
                             train_size=split_opts.train_size, test_size=split_opts.test_size,
                             valid_size=split_opts.validation_size, split_seed=split_opts.seed)

    train_dataset.selection = train_dataset.selection[:(total_images // 96) * 3]

    train_loader = DataLoader(dataset=train_dataset, num_workers=16, batch_size=train_opts.batchSize, shuffle=True)

    save_dir = os.path.join('visualisation', json_opts.model.experiment_name)
    utils.rm_and_mkdir(save_dir)

    slices = []
    masks = []

    for epoch_iter, (images, labels, indices) in tqdm(enumerate(train_loader, 1),
                                                      total=len(train_loader)):
        if epoch_iter <= total_images:

            images = images.numpy()
            labels = labels.numpy()

            for image_idx in range(images.shape[0]):
                image = np.squeeze(images[image_idx])
                label = np.squeeze(labels[image_idx])
                for slice in range(image.shape[2]):
                    if not np.max(image[..., slice]) <= 0:
                        slices.append(image[..., slice])
                        masks.append(label[..., slice])

    temp = list(zip(slices, masks))

    random.shuffle(temp)

    slices, masks = zip(*temp)
    list_index = 1
    with PdfPages(save_dir + '/augm_img_vis.pdf') as pdf:
        for page in range(nbr_pages):
            plt.figure()
            plt.figtext(.05, .9, str(json_opts.augmentation.mlebe), fontsize=4)
            idx = 1
            for slice in range(len_x * len_y):
                plt.subplot(len_y, len_x, idx)
                plt.imshow(slices[list_index], cmap='gray')
                if with_mask:
                    plt.imshow(masks[list_index], cmap='Blues', alpha=0.4)
                plt.axis('off')
                idx += 1
                list_index += 1
            pdf.savefig()
            plt.close()

    list_index = 1
    with PdfPages(save_dir + '/augm_mask_vis.pdf') as pdf:
        for page in range(nbr_pages):
            plt.figure()
            plt.figtext(.05, .9, str(json_opts.augmentation.mlebe), fontsize=4)
            idx = 1
            for slice in range(len_x * len_y):
                plt.subplot(len_y, len_x, idx)
                plt.imshow(masks[list_index], cmap='gray')
                plt.axis('off')
                idx += 1
                list_index += 1
            pdf.savefig()
            plt.close()


config_paths = ['/home/hendrik/src/MLEBE/mlebe/threed/training/configs/augm_tries/elastic.json',
                '/home/hendrik/src/MLEBE/mlebe/threed/training/configs/augm_tries/rotation.json',
                '/home/hendrik/src/MLEBE/mlebe/threed/training/configs/augm_tries/shift.json',
                '/home/hendrik/src/MLEBE/mlebe/threed/training/configs/augm_tries/flip.json',
                '/home/hendrik/src/MLEBE/mlebe/threed/training/configs/augm_tries/noise.json',
                '/home/hendrik/src/MLEBE/mlebe/threed/training/configs/try_augm.json']

# config_paths = [
#     '/home/hendrik/src/MLEBE/mlebe/threed/training/configs/try_augm.json',
# ]
for config_path in config_paths:
    aug_vis(config_path)
